In [1]:
# ============================================================================
# BUG CLASSIFICATION USING LLMS - COMPLETE IMPLEMENTATION
# ============================================================================
# This notebook implements automatic bug classification using LLMs
# with context from our manually annotated survey dataset
# ============================================================================

## SETUP AND INSTALLATION

In [2]:
import pandas as pd
import numpy as np
import json
import openai
from typing import List, Dict, Optional, Tuple
from enum import Enum
import time
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import warnings
import requests
import base64
warnings.filterwarnings('ignore')

In [3]:
!pip install openai instructor pydantic langchain scikit-learn pandas numpy seaborn matplotlib requests beautifulsoup4 PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.5/416.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 37.7 MB/s eta 0:00:00


In [4]:
import instructor
from pydantic import BaseModel, Field
from openai import OpenAI
import os
from google.colab import userdata


LLM_MODEL = 'gpt-4o-mini'
# Set up API keys
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
except:
    OPENAI_API_KEY = input("Enter your OpenAI API key: ")
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

try:
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
    os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
except:
    print("Note: GitHub token not found. Some rate limits may apply.")
    GITHUB_TOKEN = None

## DATA MODELS AND TYPES

In [5]:
class BugCategory(str, Enum):
    """Bug category enumeration for structured classification"""
    SEMANTIC = "semantic"
    MEMORY = "memory"
    CONCURRENCY = "concurrency"
    OTHER = "other"

class BugClassification(BaseModel):
    """Structured output model for bug classification"""
    category: BugCategory = Field(description="Primary bug category")
    confidence: float = Field(
        description="Confidence score between 0.0 and 1.0",
        ge=0.0, le=1.0
    )
    reasoning: str = Field(description="Explanation for the classification decision")
    secondary_categories: List[BugCategory] = Field(
        default=[],
        description="Additional relevant categories if applicable"
    )

class EnhancedIssueData(BaseModel):
    """Enhanced model for GitHub issue data with comprehensive content"""
    url: str
    title: str
    body: str
    repository: str
    issue_number: str
    labels: List[str] = Field(default=[])
    comments: List[str] = Field(default=[])
    code_snippets: List[str] = Field(default=[])
    error_traces: List[str] = Field(default=[])
    language: Optional[str] = None
    created_at: Optional[str] = None
    state: Optional[str] = None

## GITHUB ISSUE FETCHER

In [6]:
class EnhancedGitHubIssueFetcher:
    """Enhanced GitHub issue fetcher using both API and web scraping"""

    def __init__(self, github_token: Optional[str] = None):
        self.github_token = github_token
        self.github_client = Github(github_token) if github_token else Github()
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Bug-Classification-Tool/1.0',
            'Accept': 'application/vnd.github.v3+json'
        })
        if github_token:
            self.session.headers.update({'Authorization': f'token {github_token}'})

    def extract_repo_info(self, url: str) -> Tuple[str, str, str]:
        """Extract repository owner, name, and issue number from URL"""
        parts = url.replace('https://github.com/', '').split('/')
        if len(parts) >= 4 and parts[2] == 'issues':
            owner = parts[0]
            repo = parts[1]
            issue_num = parts[3]
            return owner, repo, issue_num
        raise ValueError(f"Invalid GitHub issue URL: {url}")

    def extract_code_snippets(self, text: str) -> List[str]:
        """Extract code snippets from markdown text"""
        code_blocks = re.findall(r'```[\w]*\n(.*?)\n```', text, re.DOTALL)
        inline_code = re.findall(r'`([^`\n]+)`', text)
        return code_blocks + inline_code

    def extract_error_traces(self, text: str) -> List[str]:
        """Extract error traces and stack traces from text"""
        # Common error patterns
        patterns = [
            r'(.*?Error:.*?)(?=\n\n|\n[A-Z]|\Z)',
            r'(.*?Exception:.*?)(?=\n\n|\n[A-Z]|\Z)',
            r'(Traceback.*?)(?=\n\n|\Z)',
            r'(.*?failed.*?)(?=\n\n|\n[A-Z]|\Z)',
            r'(.*?crash.*?)(?=\n\n|\n[A-Z]|\Z)'
        ]

        traces = []
        for pattern in patterns:
            matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
            traces.extend(matches)

        return [trace.strip() for trace in traces if len(trace.strip()) > 20]

    def detect_programming_language(self, repo_name: str, content: str) -> Optional[str]:
        """Detect programming language from repository name and content"""
        # Language hints from repo name
        lang_indicators = {
            'java': ['spring', 'maven', 'gradle', 'hibernate'],
            'python': ['django', 'flask', 'python', 'py'],
            'javascript': ['js', 'node', 'react', 'vue', 'angular'],
            'cpp': ['cpp', 'c++', 'cmake'],
            'c': ['linux', 'kernel', 'gcc'],
            'go': ['go', 'golang'],
            'rust': ['rust', 'cargo'],
            'php': ['php', 'laravel', 'symfony']
        }

        repo_lower = repo_name.lower()
        for lang, indicators in lang_indicators.items():
            if any(indicator in repo_lower for indicator in indicators):
                return lang

        # Language hints from content
        content_lower = content.lower()
        if 'java' in content_lower or '.java' in content_lower:
            return 'java'
        elif 'python' in content_lower or '.py' in content_lower:
            return 'python'
        elif 'javascript' in content_lower or '.js' in content_lower:
            return 'javascript'

        return None

    def fetch_issue_via_api(self, owner: str, repo: str, issue_num: str) -> Optional[EnhancedIssueData]:
        """Fetch issue using GitHub API (more reliable and comprehensive)"""
        try:
            repository = self.github_client.get_repo(f"{owner}/{repo}")
            issue = repository.get_issue(int(issue_num))

            # Get comments
            comments = []
            for comment in issue.get_comments():
                if comment.body and len(comment.body.strip()) > 10:
                    comments.append(comment.body)

            # Combine title and body for analysis
            full_content = f"{issue.title}\n\n{issue.body or ''}"
            for comment in comments[:5]:  # Limit to first 5 comments
                full_content += f"\n\n{comment}"

            # Extract structured information
            code_snippets = self.extract_code_snippets(full_content)
            error_traces = self.extract_error_traces(full_content)
            language = self.detect_programming_language(repository.name, full_content)

            return EnhancedIssueData(
                url=issue.html_url,
                title=issue.title,
                body=issue.body or "",
                repository=f"{owner}/{repo}",
                issue_number=str(issue.number),
                labels=[label.name for label in issue.labels],
                comments=comments[:10],  # Limit comments
                code_snippets=code_snippets[:5],  # Limit code snippets
                error_traces=error_traces[:3],  # Limit error traces
                language=language,
                created_at=str(issue.created_at) if issue.created_at else None,
                state=issue.state
            )

        except Exception as e:
            print(f"API fetch failed for {owner}/{repo}/issues/{issue_num}: {str(e)}")
            return None

    def fetch_issue_via_web(self, url: str) -> Optional[EnhancedIssueData]:
        """Fallback web scraping method"""
        try:
            owner, repo, issue_num = self.extract_repo_info(url)

            response = self.session.get(url)
            if response.status_code != 200:
                return None

            from bs4 import BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract title
            title_elem = soup.find('h1', class_='js-issue-title')
            if not title_elem:
                title_elem = soup.find('bdi', class_='js-issue-title')
            title = title_elem.get_text().strip() if title_elem else "No title"

            # Extract body
            body_elem = soup.find('td', class_='d-block comment-body markdown-body')
            if not body_elem:
                body_elem = soup.find('div', class_='comment-body')
            body = body_elem.get_text().strip() if body_elem else ""

            # Extract comments
            comments = []
            comment_elements = soup.find_all('td', class_='d-block comment-body markdown-body')
            for comment_elem in comment_elements[1:6]:  # Skip first (main issue), take next 5
                comment_text = comment_elem.get_text().strip()
                if len(comment_text) > 10:
                    comments.append(comment_text)

            # Extract labels
            labels = []
            label_elements = soup.find_all('a', class_='Link--muted')
            for label_elem in label_elements:
                if 'labels' in label_elem.get('href', ''):
                    labels.append(label_elem.get_text().strip())

            full_content = f"{title}\n\n{body}\n\n" + "\n\n".join(comments)
            code_snippets = self.extract_code_snippets(full_content)
            error_traces = self.extract_error_traces(full_content)
            language = self.detect_programming_language(repo, full_content)

            return EnhancedIssueData(
                url=url,
                title=title,
                body=body,
                repository=f"{owner}/{repo}",
                issue_number=issue_num,
                labels=labels,
                comments=comments,
                code_snippets=code_snippets[:5],
                error_traces=error_traces[:3],
                language=language,
                created_at=None,
                state=None
            )

        except Exception as e:
            print(f"Web fetch failed for {url}: {str(e)}")
            return None

    def extract_issue_info(self, url: str) -> Optional[EnhancedIssueData]:
        """Main method to extract issue info - tries API first, then web scraping"""
        try:
            owner, repo, issue_num = self.extract_repo_info(url)
        except ValueError as e:
            print(f"URL parsing error: {e}")
            return None

        # Try API first (more reliable)
        issue_data = self.fetch_issue_via_api(owner, repo, issue_num)

        # Fallback to web scraping if API fails
        if not issue_data:
            print(f"Falling back to web scraping for {url}")
            issue_data = self.fetch_issue_via_web(url)

        return issue_data

## LLM-BASED BUG CLASSIFIER

In [7]:
class EnhancedLLMBugClassifier:
    """Enhanced LLM-based bug classifier with better prompting and context"""

    def __init__(self, api_key: str, model: str = LLM_MODEL):
        self.client = instructor.from_openai(OpenAI(api_key=api_key))
        self.model = model
        self.examples_by_category = {}

    def set_examples(self, examples: Dict[str, List[str]]):
        """Set few-shot examples for classification"""
        self.examples_by_category = examples

    def create_enhanced_classification_prompt(self, issue: EnhancedIssueData) -> str:
        """Create comprehensive prompt with all available context"""

        # Build context sections
        context_sections = []

        # Basic info section
        context_sections.append(f"**REPOSITORY**: {issue.repository}")
        context_sections.append(f"**TITLE**: {issue.title}")

        if issue.language:
            context_sections.append(f"**LANGUAGE**: {issue.language}")

        if issue.labels:
            context_sections.append(f"**LABELS**: {', '.join(issue.labels)}")

        # Main description
        if issue.body:
            context_sections.append(f"**DESCRIPTION**:\n{issue.body[:1500]}")

        # Code snippets
        if issue.code_snippets:
            context_sections.append("**CODE SNIPPETS**:")
            for i, snippet in enumerate(issue.code_snippets[:3], 1):
                snippet_text = snippet[:300] if len(snippet) > 300 else snippet
                context_sections.append(f"{i}. ```\n{snippet_text}\n```")

        # Error traces
        if issue.error_traces:
            context_sections.append("**ERROR TRACES/LOGS**:")
            for i, trace in enumerate(issue.error_traces[:2], 1):
                trace_text = trace[:400] if len(trace) > 400 else trace
                context_sections.append(f"{i}. {trace_text}")

        # Comments (most relevant ones)
        if issue.comments:
            context_sections.append("**RELEVANT COMMENTS**:")
            for i, comment in enumerate(issue.comments[:2], 1):
                comment_text = comment[:300] if len(comment) > 300 else comment
                context_sections.append(f"{i}. {comment_text}")

        issue_context = "\n\n".join(context_sections)

        # Build few-shot examples
        examples_text = self._build_examples_text()

        prompt = f"""You are an expert software engineer specializing in bug classification.
Classify this GitHub issue into one of four categories based on the comprehensive context provided.

**CATEGORY DEFINITIONS:**

- **SEMANTIC**: API misuse, incorrect logic, specification violations, interface contract errors, wrong parameter usage, missing null checks, incorrect algorithm implementation
- **MEMORY**: Memory leaks, out-of-memory errors, resource management issues, garbage collection problems, buffer overflows, memory allocation failures
- **CONCURRENCY**: Thread safety issues, race conditions, deadlocks, synchronization problems, parallel processing errors, atomic operation failures
- **OTHER**: Configuration errors, build issues, documentation problems, environment setup, dependency conflicts, deployment issues

**CLASSIFICATION EXAMPLES:**
{examples_text}

**ISSUE TO CLASSIFY:**
{issue_context}

**CLASSIFICATION GUIDELINES:**
1. **Primary focus**: Analyze the root cause described in the issue, not just symptoms
2. **Code analysis**: Pay special attention to code snippets and error traces
3. **Context clues**: Use repository language, labels, and comments for additional context
4. **Error patterns**: Look for specific error messages that indicate category type
5. **Multi-category handling**: If multiple categories apply, choose the most fundamental cause
6. **Confidence scoring**: Base confidence on clarity of technical details provided

Provide a structured classification with detailed reasoning."""

        return prompt

    def _build_examples_text(self) -> str:
        """Build few-shot examples text"""
        examples_text = ""
        for category, examples in self.examples_by_category.items():
            category_desc = self._get_category_description(category)
            examples_text += f"\n**{category.upper()}** examples:\n"

            for i, example in enumerate(examples[:2], 1):
                repo_name = example.get('repository', 'Unknown')
                examples_text += f"  {i}. {repo_name} - {category_desc}\n"

        return examples_text

    def _get_category_description(self, category: str) -> str:
        """Get brief description for each category"""
        descriptions = {
            "semantic": "Logic/API misuse issues",
            "memory": "Memory management problems",
            "concurrency": "Threading/synchronization issues",
            "other": "Config/build/environment issues"
        }
        return descriptions.get(category, "Unknown category")

    def classify_issue(self, issue: EnhancedIssueData, max_retries: int = 3) -> Optional[BugClassification]:
        """Classify a single issue using enhanced prompting"""

        prompt = self.create_enhanced_classification_prompt(issue)

        for attempt in range(max_retries):
            try:
                classification = self.client.chat.completions.create(
                    model=self.model,
                    response_model=BugClassification,
                    messages=[
                        {
                            "role": "system",
                            "content": "You are an expert software bug classifier. Analyze the provided context comprehensively and provide accurate classifications with detailed reasoning."
                        },
                        {
                            "role": "user",
                            "content": prompt
                        }
                    ],
                    max_tokens=1000,
                )

                return classification

            except Exception as e:
                print(f"❌ Classification attempt {attempt + 1} failed: {str(e)}")
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)

        return None

    def classify_batch(self, issues: List[EnhancedIssueData], progress_callback=None) -> List[Optional[BugClassification]]:
        """Classify multiple issues with progress tracking"""
        results = []

        for i, issue in enumerate(issues):
            if progress_callback:
                progress_callback(i, len(issues))

            result = self.classify_issue(issue)
            results.append(result)

            # Rate limiting
            time.sleep(1.0)

        return results

## DATASET LOADING AND PREPROCESSING

In [8]:
class EnhancedBugDatasetLoader:
    """Enhanced dataset loader with better example selection"""

    def __init__(self, csv_path: str = '/content/bug_classification_dataset.csv'):
        self.csv_path = csv_path
        self.df = None
        self.examples_by_category = {}

    def load_dataset(self) -> Optional[pd.DataFrame]:
        """Load and validate the dataset"""
        try:
            self.df = pd.read_csv(self.csv_path)
            print(f"✅ Dataset loaded: {len(self.df)} records")

            # Validate required columns
            required_cols = ['url', 'answer']
            missing_cols = [col for col in required_cols if col not in self.df.columns]
            if missing_cols:
                print(f"❌ Missing required columns: {missing_cols}")
                return None

            # Show category distribution
            print(f"📊 Category distribution:")
            for category, count in self.df['answer'].value_counts().items():
                print(f"   {category}: {count}")

            return self.df

        except FileNotFoundError:
            print(f"❌ Dataset file not found: {self.csv_path}")
            print("Please ensure the file exists at the specified path")
            return None
        except Exception as e:
            print(f"❌ Error loading dataset: {str(e)}")
            return None

    def prepare_enhanced_examples(self, n_examples_per_category: int = 3) -> Dict[str, List[Dict]]:
        """Prepare enhanced examples with better diversity"""
        if self.df is None:
            print("❌ Dataset not loaded")
            return {}

        examples = {}
        for category in BugCategory:
            category_data = self.df[self.df['answer'] == category.value]

            if len(category_data) > 0:
                # Get diverse examples from different repositories
                if 'repo' in category_data.columns:
                    # Group by repository and take diverse samples
                    diverse_samples = category_data.groupby('repo', group_keys=False).apply(
                        lambda x: x.sample(min(len(x), 1), random_state=42)
                    ).head(n_examples_per_category)
                else:
                    # Random sample if no repo column
                    diverse_samples = category_data.sample(
                        n=min(n_examples_per_category, len(category_data)),
                        random_state=42
                    )

                examples[category.value] = []
                for _, row in diverse_samples.iterrows():
                    example = {
                        'url': row['url'],
                        'repository': row.get('repo', 'unknown'),
                        'category': category.value,
                        'annotation_time': row.get('time_spent', 30.0)
                    }
                    examples[category.value].append(example)

        self.examples_by_category = examples
        return examples

## EVALUATION AND METRICS

In [9]:
class EnhancedBugClassifierEvaluator:
    """Enhanced evaluation with more detailed metrics"""

    def __init__(self):
        self.categories = [cat.value for cat in BugCategory]

    def evaluate_predictions(self, y_true: List[str], y_pred: List[str],
                           confidences: List[float] = None,
                           detailed_results: List[BugClassification] = None) -> Dict:
        """Enhanced evaluation with confidence analysis"""

        # Filter out None predictions
        valid_indices = [i for i, pred in enumerate(y_pred) if pred is not None]
        y_true_clean = [y_true[i] for i in valid_indices]
        y_pred_clean = [y_pred[i] for i in valid_indices]

        if confidences:
            confidences_clean = [confidences[i] for i in valid_indices]
        else:
            confidences_clean = None

        # Basic metrics
        accuracy = accuracy_score(y_true_clean, y_pred_clean)

        # Classification report
        class_report = classification_report(
            y_true_clean, y_pred_clean,
            labels=self.categories,
            target_names=self.categories,
            output_dict=True,
            zero_division=0
        )

        # Confusion matrix
        cm = confusion_matrix(y_true_clean, y_pred_clean, labels=self.categories)

        results = {
            'accuracy': accuracy,
            'classification_report': class_report,
            'confusion_matrix': cm,
            'categories': self.categories,
            'total_predictions': len(y_true),
            'successful_predictions': len(y_true_clean),
            'failed_predictions': len(y_true) - len(y_true_clean)
        }

        # Confidence analysis
        if confidences_clean:
            results['avg_confidence'] = np.mean(confidences_clean)

            # Confidence by correctness
            correct_mask = [y_true_clean[i] == y_pred_clean[i] for i in range(len(y_true_clean))]
            results['confidence_by_correctness'] = {
                'correct': np.mean([confidences_clean[i] for i in range(len(confidences_clean)) if correct_mask[i]]) if any(correct_mask) else 0.0,
                'incorrect': np.mean([confidences_clean[i] for i in range(len(confidences_clean)) if not correct_mask[i]]) if not all(correct_mask) else 0.0
            }

            # Confidence distribution by category
            results['confidence_by_category'] = {}
            for cat in self.categories:
                cat_mask = [y_pred_clean[i] == cat for i in range(len(y_pred_clean))]
                if any(cat_mask):
                    results['confidence_by_category'][cat] = np.mean([
                        confidences_clean[i] for i in range(len(confidences_clean)) if cat_mask[i]
                    ])

        return results

    def print_enhanced_evaluation_report(self, results: Dict):
        """Print comprehensive evaluation results"""
        print("=" * 70)
        print("🎯 ENHANCED BUG CLASSIFICATION EVALUATION RESULTS")
        print("=" * 70)

        print(f"\n📊 **Prediction Statistics**:")
        print(f"   Total issues: {results['total_predictions']}")
        print(f"   Successful predictions: {results['successful_predictions']}")
        print(f"   Failed predictions: {results['failed_predictions']}")
        print(f"   Success rate: {results['successful_predictions']/results['total_predictions']*100:.1f}%")

        print(f"\n🎯 **Overall Accuracy**: {results['accuracy']:.3f}")

        if 'avg_confidence' in results:
            print(f"🎯 **Average Confidence**: {results['avg_confidence']:.3f}")
            conf_by_correct = results['confidence_by_correctness']
            print(f"   ✅ Correct predictions: {conf_by_correct['correct']:.3f}")
            print(f"   ❌ Incorrect predictions: {conf_by_correct['incorrect']:.3f}")

            if 'confidence_by_category' in results:
                print(f"\n📈 **Confidence by Category**:")
                for cat, conf in results['confidence_by_category'].items():
                    print(f"   {cat.upper():12}: {conf:.3f}")

        print(f"\n📈 **Per-Category Performance**:")
        class_report = results['classification_report']

        for category in self.categories:
            if category in class_report:
                metrics = class_report[category]
                print(f"   {category.upper():12} - P: {metrics['precision']:.3f} | "
                     f"R: {metrics['recall']:.3f} | F1: {metrics['f1-score']:.3f} | "
                     f"Support: {metrics['support']:3d}")

        # Macro averages
        if 'macro avg' in class_report:
            macro_avg = class_report['macro avg']
            print(f"\n🎯 **Macro Average** - P: {macro_avg['precision']:.3f} | "
                 f"R: {macro_avg['recall']:.3f} | F1: {macro_avg['f1-score']:.3f}")

    def plot_enhanced_confusion_matrix(self, results: Dict, title: str = "Bug Classification Confusion Matrix"):
        """Plot enhanced confusion matrix with better visualization"""
        plt.figure(figsize=(12, 10))

        cm = results['confusion_matrix']
        categories = results['categories']

        # Normalize confusion matrix
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        # Create heatmap
        ax = sns.heatmap(
            cm_normalized,
            annot=True,
            fmt='.2f',
            xticklabels=[cat.upper() for cat in categories],
            yticklabels=[cat.upper() for cat in categories],
            cmap='Blues',
            cbar_kws={'label': 'Normalized Frequency'}
        )

        # Add raw counts as text
        for i in range(len(categories)):
            for j in range(len(categories)):
                if cm[i, j] > 0:
                    ax.text(j + 0.5, i + 0.7, f'({cm[i, j]})',
                           ha='center', va='center', fontsize=9, color='red')

        plt.title(f'{title}\n(Normalized values with raw counts in parentheses)', fontsize=14, pad=20)
        plt.xlabel('Predicted Category', fontsize=12)
        plt.ylabel('True Category', fontsize=12)
        plt.tight_layout()
        plt.show()

## BASELINE TRADITIONAL ML CLASSIFIER

In [10]:
class BaselineBugClassifier:
    """Traditional ML baseline for comparison"""

    def __init__(self):
        self.vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
        self.classifier = LogisticRegression(random_state=42, max_iter=1000)
        self.is_trained = False

    def prepare_text(self, df: pd.DataFrame) -> List[str]:
        """Prepare text data for traditional ML"""
        texts = []
        for _, row in df.iterrows():
            # Combine URL info with repository context
            repo_name = row['repo'].split('/')[-1] if '/' in row['repo'] else row['repo']
            text = f"{repo_name} github issue"  # Simple text representation
            texts.append(text)
        return texts

    def train(self, df: pd.DataFrame):
        """Train baseline classifier"""
        texts = self.prepare_text(df)
        X = self.vectorizer.fit_transform(texts)
        y = df['answer'].values

        self.classifier.fit(X, y)
        self.is_trained = True
        print("Baseline model trained")

    def predict(self, df: pd.DataFrame) -> Tuple[List[str], List[float]]:
        """Make predictions with baseline model"""
        if not self.is_trained:
            raise ValueError("Model must be trained first")

        texts = self.prepare_text(df)
        X = self.vectorizer.transform(texts)

        predictions = self.classifier.predict(X)
        probabilities = self.classifier.predict_proba(X)
        confidences = [max(prob) for prob in probabilities]

        return predictions.tolist(), confidences

## MAIN EXECUTION PIPELINE

In [11]:
class EnhancedBugClassificationPipeline:
    """Main pipeline with enhanced issue fetching and classification"""

    def __init__(self, openai_api_key: str, github_token: Optional[str] = None):
        self.dataset_loader = EnhancedBugDatasetLoader()
        self.issue_fetcher = EnhancedGitHubIssueFetcher(github_token)
        self.classifier = EnhancedLLMBugClassifier(openai_api_key)
        self.evaluator = EnhancedBugClassifierEvaluator()

    def run_enhanced_pipeline(self, sample_size: int = 20):
        """Run enhanced pipeline with real GitHub issue fetching"""

        print("🚀 STARTING ENHANCED BUG CLASSIFICATION PIPELINE")
        print("=" * 70)

        # Step 1: Load Dataset
        print("\n📂 Step 1: Loading dataset...")
        df = self.dataset_loader.load_dataset()
        if df is None:
            return None

        # Step 2: Prepare examples
        print("\n📚 Step 2: Preparing few-shot examples...")
        examples = self.dataset_loader.prepare_enhanced_examples(n_examples_per_category=3)
        self.classifier.set_examples(examples)

        for category, category_examples in examples.items():
            print(f"   {category.upper()}: {len(category_examples)} examples")

        # Step 3: Create evaluation dataset
        print(f"\n✂️  Step 3: Preparing evaluation dataset (sample size: {sample_size})...")

        # Get a diverse sample for evaluation
        if len(df) > sample_size:
            # Stratified sampling to maintain category balance
            test_sample = df.groupby('answer', group_keys=False).apply(
                lambda x: x.sample(min(len(x), sample_size // 4), random_state=42)
            ).head(sample_size)
        else:
            test_sample = df.copy()

        print(f"   📊 Selected {len(test_sample)} issues for evaluation")
        print(f"   📊 Category distribution: {test_sample['answer'].value_counts().to_dict()}")

        # Step 4: Fetch real GitHub issue content
        print(f"\n📡 Step 4: Fetching GitHub issue content...")
        enhanced_issues = []
        fetch_failures = 0

        for i, (_, row) in enumerate(test_sample.iterrows()):
            print(f"   Fetching {i+1}/{len(test_sample)}: {row['url']}")

            issue_data = self.issue_fetcher.extract_issue_info(row['url'])
            if issue_data:
                enhanced_issues.append(issue_data)
                # Show what we extracted
                content_info = []
                if issue_data.body and len(issue_data.body.strip()) > 0:
                    content_info.append("description")
                if issue_data.comments:
                    content_info.append(f"{len(issue_data.comments)} comments")
                if issue_data.code_snippets:
                    content_info.append(f"{len(issue_data.code_snippets)} code snippets")
                if issue_data.error_traces:
                    content_info.append(f"{len(issue_data.error_traces)} error traces")
                if issue_data.labels:
                    content_info.append(f"{len(issue_data.labels)} labels")

                print(f"      ✅ Extracted: {', '.join(content_info) if content_info else 'basic info only'}")
            else:
                enhanced_issues.append(None)
                fetch_failures += 1
                print(f"      ❌ Failed to fetch content")

            # Rate limiting
            time.sleep(0.5)

        print(f"   📊 Successfully fetched: {len(enhanced_issues) - fetch_failures}/{len(enhanced_issues)}")
        print(f"   📊 Fetch failure rate: {fetch_failures/len(enhanced_issues)*100:.1f}%")

        # Step 5: Enhanced LLM Classification
        print(f"\n🧠 Step 5: Enhanced LLM classification...")

        def progress_callback(current, total):
            print(f"      Progress: {current + 1}/{total} ({(current + 1)/total*100:.1f}%)")

        # Filter out failed fetches
        valid_issues = [(issue, i) for i, issue in enumerate(enhanced_issues) if issue is not None]
        valid_test_sample = test_sample.iloc[[i for issue, i in valid_issues]].reset_index(drop=True)
        valid_enhanced_issues = [issue for issue, i in valid_issues]

        print(f"   📊 Classifying {len(valid_enhanced_issues)} successfully fetched issues...")

        classification_results = self.classifier.classify_batch(
            valid_enhanced_issues,
            progress_callback
        )

        # Step 6: Extract predictions and evaluate
        print(f"\n📈 Step 6: Evaluation and analysis...")

        true_labels = valid_test_sample['answer'].tolist()
        llm_predictions = [r.category.value if r else "other" for r in classification_results]
        llm_confidences = [r.confidence if r else 0.0 for r in classification_results]

        # Evaluate results
        evaluation_results = self.evaluator.evaluate_predictions(
            true_labels, llm_predictions, llm_confidences, classification_results
        )

        # Print detailed results
        print(f"\n🎯 ENHANCED LLM CLASSIFIER RESULTS:")
        self.evaluator.print_enhanced_evaluation_report(evaluation_results)

        # Step 7: Detailed Analysis
        print(f"\n🔍 Step 7: Detailed prediction analysis...")
        self._print_detailed_analysis(
            valid_enhanced_issues, true_labels, llm_predictions,
            llm_confidences, classification_results
        )

        # Step 8: Visualization
        print(f"\n📊 Step 8: Generating visualizations...")
        self.evaluator.plot_enhanced_confusion_matrix(
            evaluation_results,
            "Enhanced LLM Bug Classifier - Confusion Matrix"
        )

        # Additional analysis plots
        self._plot_confidence_analysis(llm_confidences, true_labels, llm_predictions)
        self._plot_category_performance(evaluation_results)

        print(f"\n🎉 ENHANCED PIPELINE COMPLETED SUCCESSFULLY!")

        return {
            'evaluation_results': evaluation_results,
            'test_data': valid_test_sample,
            'enhanced_issues': valid_enhanced_issues,
            'predictions': {
                'true': true_labels,
                'predicted': llm_predictions,
                'confidences': llm_confidences
            },
            'classification_details': classification_results
        }

    def _print_detailed_analysis(self, issues: List[EnhancedIssueData],
                               true_labels: List[str], predictions: List[str],
                               confidences: List[float],
                               classification_results: List[BugClassification]):
        """Print detailed analysis of predictions"""

        print("=" * 70)
        print("🔍 DETAILED PREDICTION ANALYSIS")
        print("=" * 70)

        # Group by prediction accuracy
        correct_predictions = []
        incorrect_predictions = []

        for i, (issue, true_label, pred, conf, result) in enumerate(
            zip(issues, true_labels, predictions, confidences, classification_results)
        ):
            analysis = {
                'index': i,
                'issue': issue,
                'true_label': true_label,
                'predicted': pred,
                'confidence': conf,
                'result': result,
                'correct': true_label == pred
            }

            if analysis['correct']:
                correct_predictions.append(analysis)
            else:
                incorrect_predictions.append(analysis)

        # Show some correct predictions
        print(f"\n✅ CORRECT PREDICTIONS (showing top 3 by confidence):")
        correct_sorted = sorted(correct_predictions, key=lambda x: x['confidence'], reverse=True)
        for analysis in correct_sorted[:3]:
            self._print_prediction_details(analysis)

        # Show some incorrect predictions
        print(f"\n❌ INCORRECT PREDICTIONS (showing top 3 by confidence):")
        incorrect_sorted = sorted(incorrect_predictions, key=lambda x: x['confidence'], reverse=True)
        for analysis in incorrect_sorted[:3]:
            self._print_prediction_details(analysis)

        # Show low confidence predictions
        all_predictions = correct_predictions + incorrect_predictions
        low_confidence = sorted([p for p in all_predictions if p['confidence'] < 0.6],
                              key=lambda x: x['confidence'])

        if low_confidence:
            print(f"\n⚠️  LOW CONFIDENCE PREDICTIONS (< 0.6, showing first 2):")
            for analysis in low_confidence[:2]:
                self._print_prediction_details(analysis)

    def _print_prediction_details(self, analysis: Dict):
        """Print details for a single prediction"""
        issue = analysis['issue']
        result = analysis['result']
        status = "✅" if analysis['correct'] else "❌"

        print(f"\n{status} Issue: {issue.repository}/issues/{issue.issue_number}")
        print(f"   📋 Title: {issue.title[:80]}...")
        print(f"   🎯 True: {analysis['true_label']} | Predicted: {analysis['predicted']} | Confidence: {analysis['confidence']:.3f}")

        if result and result.reasoning:
            reasoning = result.reasoning[:150] + "..." if len(result.reasoning) > 150 else result.reasoning
            print(f"   💭 Reasoning: {reasoning}")

        # Show extracted features
        features = []
        if issue.language:
            features.append(f"Language: {issue.language}")
        if issue.labels:
            features.append(f"Labels: {', '.join(issue.labels[:3])}")
        if issue.code_snippets:
            features.append(f"Code snippets: {len(issue.code_snippets)}")
        if issue.error_traces:
            features.append(f"Error traces: {len(issue.error_traces)}")
        if issue.comments:
            features.append(f"Comments: {len(issue.comments)}")

        if features:
            print(f"   🔍 Features: {' | '.join(features)}")

    def _plot_confidence_analysis(self, confidences: List[float],
                                true_labels: List[str], predictions: List[str]):
        """Plot confidence distribution and analysis"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

        # Confidence distribution
        ax1.hist(confidences, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
        ax1.set_xlabel('Confidence Score')
        ax1.set_ylabel('Frequency')
        ax1.set_title('Distribution of Prediction Confidence')
        ax1.axvline(np.mean(confidences), color='red', linestyle='--',
                   label=f'Mean: {np.mean(confidences):.3f}')
        ax1.legend()

        # Confidence by correctness
        correct_mask = [true_labels[i] == predictions[i] for i in range(len(true_labels))]
        correct_conf = [confidences[i] for i in range(len(confidences)) if correct_mask[i]]
        incorrect_conf = [confidences[i] for i in range(len(confidences)) if not correct_mask[i]]

        ax2.boxplot([correct_conf, incorrect_conf], labels=['Correct', 'Incorrect'])
        ax2.set_ylabel('Confidence Score')
        ax2.set_title('Confidence by Prediction Accuracy')

        # Confidence by category
        categories = list(set(predictions))
        conf_by_cat = {cat: [confidences[i] for i in range(len(confidences))
                            if predictions[i] == cat] for cat in categories}

        ax3.boxplot([conf_by_cat[cat] for cat in categories], labels=categories)
        ax3.set_ylabel('Confidence Score')
        ax3.set_title('Confidence Distribution by Predicted Category')
        ax3.tick_params(axis='x', rotation=45)

        # Confidence vs Accuracy scatter
        accuracy_by_conf_bin = []
        conf_bins = np.linspace(0, 1, 11)
        bin_centers = []

        for i in range(len(conf_bins) - 1):
            bin_mask = [(confidences[j] >= conf_bins[i] and confidences[j] < conf_bins[i+1])
                       for j in range(len(confidences))]
            if any(bin_mask):
                bin_accuracy = np.mean([correct_mask[j] for j in range(len(correct_mask)) if bin_mask[j]])
                accuracy_by_conf_bin.append(bin_accuracy)
                bin_centers.append((conf_bins[i] + conf_bins[i+1]) / 2)

        if bin_centers:
            ax4.scatter(bin_centers, accuracy_by_conf_bin, alpha=0.7, s=60)
            ax4.plot(bin_centers, accuracy_by_conf_bin, 'r--', alpha=0.5)
            ax4.set_xlabel('Confidence Score')
            ax4.set_ylabel('Accuracy')
            ax4.set_title('Accuracy vs Confidence Score')
            ax4.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    def _plot_category_performance(self, evaluation_results: Dict):
        """Plot detailed category performance analysis"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        class_report = evaluation_results['classification_report']
        categories = evaluation_results['categories']

        # Per-category metrics
        metrics = ['precision', 'recall', 'f1-score']
        metric_values = {metric: [] for metric in metrics}

        for category in categories:
            if category in class_report:
                for metric in metrics:
                    metric_values[metric].append(class_report[category][metric])
            else:
                for metric in metrics:
                    metric_values[metric].append(0.0)

        x = np.arange(len(categories))
        width = 0.25

        for i, metric in enumerate(metrics):
            ax1.bar(x + i * width, metric_values[metric], width,
                   label=metric.capitalize(), alpha=0.8)

        ax1.set_xlabel('Categories')
        ax1.set_ylabel('Score')
        ax1.set_title('Per-Category Performance Metrics')
        ax1.set_xticks(x + width)
        ax1.set_xticklabels([cat.upper() for cat in categories])
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Support (number of samples) per category
        support_values = []
        for category in categories:
            if category in class_report:
                support_values.append(class_report[category]['support'])
            else:
                support_values.append(0)

        ax2.bar(categories, support_values, alpha=0.7, color='lightcoral')
        ax2.set_xlabel('Categories')
        ax2.set_ylabel('Number of Samples')
        ax2.set_title('Dataset Support per Category')
        ax2.set_xticklabels([cat.upper() for cat in categories])

        # Add value labels on bars
        for i, v in enumerate(support_values):
            ax2.text(i, v + 0.1, str(v), ha='center', va='bottom')

        plt.tight_layout()
        plt.show()

## UTILITY FUNCTIONS FOR PRODUCTION USE

In [12]:
def classify_single_issue(url: str, openai_api_key: str,
                         github_token: Optional[str] = None) -> Optional[BugClassification]:
    """Classify a single GitHub issue URL with enhanced content extraction"""

    print(f"🔍 Analyzing issue: {url}")

    # Initialize components
    fetcher = EnhancedGitHubIssueFetcher(github_token)
    classifier = EnhancedLLMBugClassifier(openai_api_key)

    # Set up basic examples (in production, load from your dataset)
    basic_examples = {
        'semantic': [{'repository': 'spring-boot', 'category': 'semantic', 'annotation_time': 45.0}],
        'memory': [{'repository': 'elasticsearch', 'category': 'memory', 'annotation_time': 52.0}],
        'concurrency': [{'repository': 'netty', 'category': 'concurrency', 'annotation_time': 23.0}],
        'other': [{'repository': 'gradle', 'category': 'other', 'annotation_time': 15.0}]
    }
    classifier.set_examples(basic_examples)

    # Fetch enhanced issue data
    issue_data = fetcher.extract_issue_info(url)
    if not issue_data:
        print("❌ Failed to fetch issue data")
        return None

    print(f"📋 Issue: {issue_data.title}")
    print(f"🏠 Repository: {issue_data.repository}")

    # Show extracted features
    features = []
    if issue_data.language:
        features.append(f"Language: {issue_data.language}")
    if issue_data.labels:
        features.append(f"Labels: {len(issue_data.labels)}")
    if issue_data.code_snippets:
        features.append(f"Code: {len(issue_data.code_snippets)} snippets")
    if issue_data.error_traces:
        features.append(f"Errors: {len(issue_data.error_traces)} traces")
    if issue_data.comments:
        features.append(f"Comments: {len(issue_data.comments)}")

    if features:
        print(f"🔍 Extracted features: {' | '.join(features)}")

    # Classify
    result = classifier.classify_issue(issue_data)

    if result:
        print(f"\n🎯 Classification: {result.category.value}")
        print(f"📊 Confidence: {result.confidence:.3f}")
        print(f"💭 Reasoning: {result.reasoning}")

        if result.secondary_categories:
            print(f"🔄 Secondary categories: {[cat.value for cat in result.secondary_categories]}")

    return result

def batch_classify_with_enhanced_features(csv_file: str, openai_api_key: str,
                                        github_token: Optional[str] = None,
                                        output_file: str = "enhanced_classification_results.csv",
                                        max_issues: int = 100):
    """Enhanced batch classification with comprehensive feature extraction"""

    print(f"🚀 ENHANCED BATCH CLASSIFICATION")
    print("=" * 50)

    # Load input data
    try:
        df = pd.read_csv(csv_file)
        if 'url' not in df.columns:
            print("❌ CSV must contain 'url' column")
            return None
    except Exception as e:
        print(f"❌ Error loading CSV: {e}")
        return None

    # Limit processing if requested
    if len(df) > max_issues:
        print(f"⚠️ Limiting to first {max_issues} issues (from {len(df)} total)")
        df = df.head(max_issues)

    # Initialize components
    fetcher = EnhancedGitHubIssueFetcher(github_token)
    classifier = EnhancedLLMBugClassifier(openai_api_key)

    # Set basic examples
    basic_examples = {
        'semantic': [{'repository': 'spring-boot', 'category': 'semantic', 'annotation_time': 45.0}],
        'memory': [{'repository': 'elasticsearch', 'category': 'memory', 'annotation_time': 52.0}],
        'concurrency': [{'repository': 'netty', 'category': 'concurrency', 'annotation_time': 23.0}],
        'other': [{'repository': 'gradle', 'category': 'other', 'annotation_time': 15.0}]
    }
    classifier.set_examples(basic_examples)

    results = []
    fetch_failures = 0
    classification_failures = 0

    print(f"📡 Processing {len(df)} issues...")

    for i, row in df.iterrows():
        print(f"Progress: {i+1}/{len(df)} - {row['url']}")

        # Fetch enhanced issue data
        issue_data = fetcher.extract_issue_info(row['url'])

        if not issue_data:
            fetch_failures += 1
            print(f"   ❌ Failed to fetch issue content")

            # Create minimal result for failed fetch
            result = {
                'url': row['url'],
                'repository': 'unknown',
                'title': 'Failed to fetch',
                'fetch_success': False,
                'predicted_category': 'unknown',
                'confidence': 0.0,
                'reasoning': 'Failed to fetch issue content',
                'language': None,
                'num_labels': 0,
                'num_comments': 0,
                'num_code_snippets': 0,
                'num_error_traces': 0,
                'has_description': False
            }
        else:
            print(f"   ✅ Fetched content")

            # Classify
            classification = classifier.classify_issue(issue_data)

            if classification:
                print(f"   🎯 Classified as: {classification.category.value} (confidence: {classification.confidence:.3f})")
            else:
                classification_failures += 1
                print(f"   ❌ Classification failed")

            # Create comprehensive result
            result = {
                'url': row['url'],
                'repository': issue_data.repository,
                'title': issue_data.title,
                'fetch_success': True,
                'predicted_category': classification.category.value if classification else 'unknown',
                'confidence': classification.confidence if classification else 0.0,
                'reasoning': classification.reasoning if classification else 'Classification failed',
                'secondary_categories': ','.join([cat.value for cat in classification.secondary_categories]) if classification and classification.secondary_categories else '',
                'language': issue_data.language,
                'labels': '|'.join(issue_data.labels) if issue_data.labels else '',
                'num_labels': len(issue_data.labels),
                'num_comments': len(issue_data.comments),
                'num_code_snippets': len(issue_data.code_snippets),
                'num_error_traces': len(issue_data.error_traces),
                'has_description': len(issue_data.body.strip()) > 0 if issue_data.body else False,
                'description_length': len(issue_data.body) if issue_data.body else 0,
                'state': issue_data.state,
                'created_at': issue_data.created_at
            }

        # Add original columns
        for col in df.columns:
            if col not in result:
                result[f'original_{col}'] = row[col]

        results.append(result)

        # Rate limiting
        time.sleep(1.0)

    # Create results DataFrame
    results_df = pd.DataFrame(results)

    # Save results
    results_df.to_csv(output_file, index=False)

    # Print summary
    print(f"\n📊 BATCH CLASSIFICATION SUMMARY:")
    print(f"   Total issues processed: {len(df)}")
    print(f"   Successful fetches: {len(df) - fetch_failures}")
    print(f"   Fetch failures: {fetch_failures}")
    print(f"   Classification failures: {classification_failures}")
    print(f"   Overall success rate: {(len(df) - fetch_failures - classification_failures) / len(df) * 100:.1f}%")

    # Category distribution
    successful_classifications = results_df[results_df['predicted_category'] != 'unknown']
    if len(successful_classifications) > 0:
        print(f"\n📈 Predicted category distribution:")
        for category, count in successful_classifications['predicted_category'].value_counts().items():
            print(f"   {category}: {count}")

    print(f"\n✅ Results saved to: {output_file}")

    return results_df

## MAIN EXECUTION

In [13]:
def main():
    """Main function to run the enhanced bug classification system"""

    print("🚀 ENHANCED BUG CLASSIFICATION SYSTEM")
    print("=" * 70)

    # Check for dataset
    import os
    dataset_path = '/content/bug_classification_dataset.csv'

    if not os.path.exists(dataset_path):
        print(f"❌ Dataset not found at: {dataset_path}")
        print("Please upload your dataset file first using the upload function below")
        return None

    # Initialize enhanced pipeline
    try:
        pipeline = EnhancedBugClassificationPipeline(OPENAI_API_KEY, GITHUB_TOKEN)
        print("✅ Pipeline initialized successfully")
    except Exception as e:
        print(f"❌ Failed to initialize pipeline: {e}")
        return None

    # Run enhanced pipeline
    try:
        print(f"\n🏃 Starting enhanced pipeline execution...")
        results = pipeline.run_enhanced_pipeline(sample_size=15)  # Smaller sample for demo

        if results:
            print(f"\n🎊 SUCCESS! Enhanced bug classification completed!")
            print("=" * 70)

            # Show summary statistics
            eval_results = results['evaluation_results']
            print(f"\n📊 FINAL SUMMARY:")
            print(f"   Overall accuracy: {eval_results['accuracy']:.3f}")
            print(f"   Average confidence: {eval_results.get('avg_confidence', 0):.3f}")
            print(f"   Successful predictions: {eval_results['successful_predictions']}")
            print(f"   Failed predictions: {eval_results['failed_predictions']}")

            return results
        else:
            print(f"❌ Pipeline execution failed")
            return None

    except Exception as e:
        print(f"❌ Pipeline execution error: {e}")
        import traceback
        traceback.print_exc()
        return None

# File upload function for Colab
def upload_dataset():
    """Upload dataset file in Google Colab"""
    from google.colab import files

    print("📁 Upload your bug_classification_dataset.csv file:")
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f"✅ Uploaded: {filename} ({len(uploaded[filename])} bytes)")

    # Move to expected location
    import shutil
    for filename in uploaded.keys():
        if filename.endswith('.csv'):
            shutil.move(filename, '/content/bug_classification_dataset.csv')
            print(f"📂 Moved {filename} to /content/bug_classification_dataset.csv")
            print("🎉 Dataset ready! You can now run main()")
            break
    else:
        print("⚠️ Please upload a CSV file containing your dataset")

# Demo functions
def demo_single_classification():
    """Demo single issue classification"""
    test_url = "https://github.com/spring-projects/spring-boot/issues/29321"

    result = classify_single_issue(test_url, OPENAI_API_KEY, GITHUB_TOKEN)

    if result:
        print("\n✅ Single classification demo completed!")
    else:
        print("\n❌ Single classification demo failed")

def demo_batch_classification():
    """Demo batch classification with a few URLs"""
    import pandas as pd

    # Create demo CSV
    demo_urls = [
        "https://github.com/elastic/elasticsearch/issues/82391",
        "https://github.com/netty/netty/issues/11806",
        "https://github.com/spring-projects/spring-boot/issues/29321"
    ]

    demo_df = pd.DataFrame({'url': demo_urls})
    demo_df.to_csv('/content/demo_issues.csv', index=False)

    print("🧪 Running batch classification demo...")
    results = batch_classify_with_enhanced_features(
        '/content/demo_issues.csv',
        OPENAI_API_KEY,
        GITHUB_TOKEN,
        '/content/demo_results.csv',
        max_issues=3
    )

    if results is not None:
        print("\n✅ Batch classification demo completed!")
        print(f"Results saved to: /content/demo_results.csv")
        return results
    else:
        print("\n❌ Batch classification demo failed")

## RUN THE SYSTEM

In [14]:
if __name__ == "__main__":
    print("🚀 ENHANCED BUG CLASSIFICATION SYSTEM - READY!")
    print("=" * 70)
    print("\n📋 Available functions:")
    print("1. upload_dataset() - Upload your training dataset")
    print("2. main() - Run the complete enhanced pipeline")
    print("3. demo_single_classification() - Test single issue classification")
    print("4. demo_batch_classification() - Test batch classification")
    print("5. classify_single_issue(url, api_key, github_token) - Classify any GitHub issue")
    print("\n💡 Start by running: upload_dataset()")

🚀 ENHANCED BUG CLASSIFICATION SYSTEM - READY!

📋 Available functions:
1. upload_dataset() - Upload your training dataset
2. main() - Run the complete enhanced pipeline
3. demo_single_classification() - Test single issue classification
4. demo_batch_classification() - Test batch classification
5. classify_single_issue(url, api_key, github_token) - Classify any GitHub issue

💡 Start by running: upload_dataset()


In [15]:
upload_dataset()

📁 Upload your bug_classification_dataset.csv file:


Saving bug_classification_dataset.csv to bug_classification_dataset.csv
✅ Uploaded: bug_classification_dataset.csv (50663 bytes)
📂 Moved bug_classification_dataset.csv to /content/bug_classification_dataset.csv
🎉 Dataset ready! You can now run main()


In [16]:
main()

🚀 ENHANCED BUG CLASSIFICATION SYSTEM
❌ Failed to initialize pipeline: name 'Github' is not defined
